# Overview

Modified version of dna-barcodes by David Feldman for Jupyter notebook:
    https://github.com/feldman4/dna-barcodes/
    
Includes functions from his optically pooled screen repo:
    https://github.com/feldman4/OpticalPooledScreens/blob/master/ops/pool_design.py
    
Example where this script is used to generate barcodes for LAMP-seq below.

# To Do

Implement bin reduction by targeting bins with lowest remaining indices for speed-up
Understand khash implementation

In [1]:
#Check for/install dependencies, may need to restart kernel after
%pip install --user tqdm
%pip install --user multiprocess

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import useful general  libraries
from collections import Counter, defaultdict
from itertools import product
import Levenshtein
import numpy as np
import pandas as pd
import scipy.sparse
import regex as re
from tqdm.notebook import tnrange,tqdm_notebook, tqdm

#Packages for parallelization:
import os
import multiprocessing.pool
from multiprocessing import Pool 
import time
import psutil
from functools import partial

In [57]:
#All parameters (variant of function for JSB)

#Parameters for this run:
barcode_length = 7 #Barcode length 
desired_Levenshtein_distance = 3 #Minimum pairwise edit distance
sequence_to_exclude = "GTCC" #Sequence to exclude i.e. RC of 3' end of primer of length x. Leave as "" if none.
left_seq = "TCAG" #Sequence of length x to the left of BC
right_seq = "CCAA" #Sequence of length x to the right of BC
max_MM_to_exclude = 1 #Maximum number of MMs to sequence_to_exclude for filtering
file_suffix = "PrimerSet4JSB" #String to append as identifier for output .csv

#Global parameters:
barcode_length_cap = 11
min_GC_percent = 30
max_GC_percent = 70
max_homopolymer_length = 4
seq_input_number_limit = 0 #limit to number of sequences input to design process, 0=no limit
max_pairs_checked = int(1e6) #maximum number of barcode pairs to verify edit distance

In [58]:
#All parameters

#Parameters for this run:
barcode_length = 7 #Barcode length 
desired_Levenshtein_distance = 3 #Minimum pairwise edit distance
regex_for_exclusion = "(?:(GTC$|GTCC|GT$|^TCC)){s<2}" #Regex for excluding barcodes, re syntax, "" for none
file_suffix = "PrimerSet4" #String to append as identifier for output .csv

#Global parameters:
barcode_length_cap = 11
min_GC_percent = 30
max_GC_percent = 70
max_homopolymer_length = 4
seq_input_number_limit = 0 #limit to number of sequences input to design process, 0=no limit
max_pairs_checked = int(1e6) #maximum number of barcode pairs to verify edit distance

## Helper Functions

In [4]:
#Progress bar using starmap

def istarmap(self, func, iterable, chunksize=1):
    """ https://stackoverflow.com/questions/57354700/starmap-combined-with-tqdm """
    
    mpp = multiprocessing.pool

    if self._state != mpp.RUN:
        raise ValueError("Pool not running")

    if chunksize < 1:
        raise ValueError(
            "Chunksize must be 1+, not {0:n}".format(
                chunksize))

    task_batches = mpp.Pool._get_tasks(func, iterable, chunksize)
    result = mpp.IMapIterator(self._cache)
    self._taskqueue.put(
        (
            self._guarded_task_generation(result._job,
                                          mpp.starmapstar,
                                          task_batches),
            result._set_length
        ))
    return (item for chunk in result for item in chunk)


multiprocessing.pool.Pool.istarmap = istarmap

In [5]:
# Basic string functions

def generate_all_barcodes(n, lengthcap):
    ''' Generates all barcodes of length n, capped at length lengthcap '''
    if n > lengthcap:
        raise ValueError
        
    return [''.join(x) for x in product(*(n*[list('ACTG')]))]

def calculate_gc(s):
    ''' Calculates GC proportion of a string s'''
    s = s.upper()
    return (s.count('G') + s.count('C')) / len(s)

def has_homopolymer(x, n):
    ''' Checks if homopolymer of length n found in string x'''
    a = 'A'*n in x
    t = 'T'*n in x
    g = 'G'*n in x
    c = 'C'*n in x
    return (a or t or g or c)


In [6]:
# Hashing and set reduction
# Unmodified from original, room for improvement

def khash(s, k):
    """
    Divide a string s into substrings suitable for checking edit distance of k.
    Two strings of the same length with Levenshtein edit distance < k will share at least one substring. 
    """
    n = len(s)
    window = int(np.ceil((n - k) / float(k)))
    s = s + s
    arr = []
    for i in range(n):
        for j in (0, 1):
            arr += [((i + j) % n, s[i:i+window])]
    return arr

def build_khash(xs, k, return_dict=False):
    D = defaultdict(list)
    for x in xs:
        for h in khash(x, k):
             D[h].append(x)

    D = {k: sorted(set(v)) for k,v in D.items()}
    if return_dict:
        return D
    else:
        hash_buckets = list(D.values())
        return hash_buckets

def sparse_dist(hash_buckets, k, distance=None, progress=None):
    """ Entries less than k only. """
    if distance is None:
        distance = Levenshtein.distance
    if progress is None:
        progress = lambda x: x
    D = {}
    for xs in progress(hash_buckets):
        for i, a in enumerate(xs):
            for b in xs[i+1:]:
                d = distance(a,b)
                if d < k:
                    key = tuple(sorted((a,b)))
                    D[key] = d
    return D


def sparse_view(xs, D, symmetric=True):
    """ string barcodes """
    assert len(xs) == len(set(xs))
    mapper = {x: i for i, x in enumerate(xs)}
    f = lambda x: mapper[x]
    if len(D) == 0:
        i, j, data = [], [], []
    else:
        i, j, data = zip(*[(f(a), f(b), v) for (a, b), v in D.items()])
        # sparse matrix uses zero for missing values
        data = np.array(data) >= 0
        i = np.array(i)
        j = np.array(j)

    n = len(xs)
    cm = scipy.sparse.coo_matrix((data, (i, j)), shape=(n, n))

    if symmetric:
        cm = (cm + cm.T).tocsr()
        
    return cm


def maxy_clique_groups(cm, group_ids, verbose=False):
    """
    Prioritizes groups with the fewest selected barcodes.
    Prioritizing groups with the fewest remaining barcodes could give
    better results.
    """

    # counts => group_id
    d1 = defaultdict(set)
    for id_, counts in Counter(group_ids).items():
        d1[counts] |= {id_}

    # group_id => indices
    d2 = defaultdict(list)
    for i, id_ in enumerate(group_ids):
        d2[id_] += [i]
    # .pop() takes from the end of the list
    d2 = {k: v[::-1] for k,v in d2.items()}

    # group_id => # selected
    d3 = Counter()

    selected = []
    available = np.array(range(len(group_ids)))

    while d1:
        if verbose and (len(selected) % 1000) == 0:
            print(len(selected))
    #     assert cm[selected, :][:, selected].sum() == 0

        # pick a group_id from the lowest bin
        count = min(d1.keys())
        id_ = d1[count].pop()

        # remove bin if empty
        if len(d1[count]) == 0:
            d1.pop(count)

        # discard indices until we find a new one
        index = None
        while d2[id_]:
            index = d2[id_].pop()
            # approach 1: check for conflict every time
            # cm[index, selected].sum() == 0
            # approach 2: keep an array of available indices
            if index in available:
                break
        else:
            index = None

        # keep index
        if index:
            selected.append(index)
            d3[id_] += 1
            available = available[available != index]
            # get rid of incompatible barcodes
            remove = cm[index, available].indices
            mask = np.ones(len(available), dtype=bool)
            mask[remove] = False
            available = available[mask]

        # move group_id to another bin
        n = len(d2[id_])
        if n > 0:
            d1[n] |= {id_}

    return selected

In [7]:
# Levenshtein distance validation

def check_barcode_set(barcodes, k, distance='Levenshtein', max_to_check=1e6):
    """
    Returns list of barcode pairs that fail to satisfy distance k.
    """
    if distance == 'Levenshtein':
        distance = Levenshtein.distance
    failures = []
    for a in barcodes:
        for b in barcodes:
            d = distance(a, b)
            max_to_check -= 1
            if a != b and d < k:
                failures += [(a, b, d)]
            if max_to_check == 0:
                return failures
    return failures


def handle_failures(barcodes, k, max_to_check=1e6, num_failures_to_print=10):
    max_to_check = int(max_to_check)
    
    print('Validating barcodes...')
    failures = check_barcode_set(barcodes, k, max_to_check=max_to_check)
    if failures:
        print('!! Failures detected !!')
        
        for failure in range(num_failures_to_print):
            print('{} {} distance={}'.format(*failure))
        if len(failures) > num_failures_to_print:
            print('...')

    else:
        if max_to_check > (len(barcodes)**2):
            print('All barcodes passed!')
        else:
            print(f'Checked {max_to_check:,} barcode pairs, all passed!')

    return failures

## Main Functions

In [32]:
# Parallelized with max cores
    
def create_barcode_set(n, k, homopolymer, gc_min, gc_max, lengthcap = 11, exclude=None,
    limit=None):

    df_bcs = (pd.DataFrame({'barcode': generate_all_barcodes(n, lengthcap)})
     .assign(gc=lambda x: x['barcode'].apply(calculate_gc)))

    print(f'Generated {len(df_bcs)} barcodes of length {n}')

    barcodes = (df_bcs
     .query('@gc_min < gc < @gc_max')
     .loc[lambda x: ~(x['barcode'].apply(lambda y:
        has_homopolymer(y, homopolymer)))]
     ['barcode'].pipe(list))

    print(f'Retained {len(barcodes)} barcodes after '
        'filtering for GC content and homopolymers')

    if exclude is not None:
        pat = re.compile(exclude)
        barcodes = [bc for bc in barcodes if not pat.findall(bc)]
        print(f'Retained {len(barcodes)} barcodes after removing matches '
            f'to "{exclude}"')

    if limit:
        rs = np.random.RandomState(0)
        barcodes = rs.choice(barcodes, limit, replace=False)
    print('Assigning barcodes to hash buckets...')
    hash_buckets = build_khash(barcodes, k)
    print('Calculating distances within buckets...')

    with Pool(psutil.cpu_count(logical=False)) as p:
        do = lambda x: sparse_dist(*x)
        work = [([x], k, None) for x in hash_buckets]
        D = {}
        with tqdm(total=len(hash_buckets)) as pbar:
            for d in p.istarmap(sparse_dist, work):
                pbar.update()
                D.update(d)
        p.close()

    cm = sparse_view(barcodes, D)

    print(f'Selecting barcodes with minimum edit distance {k}...')
    group_ids = [0] * len(barcodes)
    selected = maxy_clique_groups(cm, group_ids)

    selected_barcodes = [barcodes[x] for x in selected]
    print(f'Selected {len(selected_barcodes)} barcodes')
    
    return (df_bcs
            .query('barcode == @selected_barcodes')
            .assign(n=n, k=k, homopolymer=homopolymer, 
                gc_min=gc_min, gc_max=gc_max))

In [9]:
def DNA_Barcode_Generator(barcode_length, 
                          desired_Levenshtein_distance, 
                          regex_for_exclusion,
                          file_suffix,
                          barcode_length_cap = 11, 
                          min_GC_percent = 30, 
                          max_GC_percent = 70,
                          max_homopolymer_length = 4,
                          seq_input_number_limit = 0,
                          max_pairs_checked = int(1e6)):

    #Set variables
    limit = None if seq_input_number_limit == 0 else seq_input_number_limit
    exclude = None if regex_for_exclusion == '' else regex_for_exclusion
    
    #Run main function
    df_bcs = create_barcode_set(barcode_length, desired_Levenshtein_distance, max_homopolymer_length, min_GC_percent/100,
                       max_GC_percent/100, barcode_length_cap, exclude, limit)
    
    #Validate output
    failures = handle_failures(df_bcs['barcode'], desired_Levenshtein_distance, max_pairs_checked)

    #Export barcodes to .csv
    failure_tag = '.failure' if failures else ''
    filename = f'barcodes_n{barcode_length}_k{desired_Levenshtein_distance}{failure_tag}_{file_suffix}.csv'
    df_bcs.to_csv(filename, index=None)

## Main Functions (JSB variant)

In [50]:
# Parallelized with max cores  

def create_barcode_set_for_JSB(n, k, homopolymer, gc_min, gc_max, left_seq, right_seq, max_MM_to_exclude, lengthcap = 11, exclude=None,
    limit=None):

    df_bcs = (pd.DataFrame({'barcode': generate_all_barcodes(n, lengthcap)})
     .assign(gc=lambda x: x['barcode'].apply(calculate_gc)))

    print(f'Generated {len(df_bcs)} barcodes of length {n}')

    barcodes = (df_bcs
     .query('@gc_min < gc < @gc_max')
     .loc[lambda x: ~(x['barcode'].apply(lambda y:
        has_homopolymer(y, homopolymer)))]
     ['barcode'].pipe(list))

    print(f'Retained {len(barcodes)} barcodes after '
        'filtering for GC content and homopolymers')

    if exclude is not None:
        exclude = f'({exclude}){{s<{max_MM_to_exclude+1}}}'
        pat = re.compile(exclude)
        barcodes = [bc for bc in barcodes if not pat.findall(left_seq+bc+right_seq)]
        print(f'Retained {len(barcodes)} barcodes after removing matches '
            f'to "{exclude}"')
        
    if limit:
        rs = np.random.RandomState(0)
        barcodes = rs.choice(barcodes, limit, replace=False)
    print('Assigning barcodes to hash buckets...')
    hash_buckets = build_khash(barcodes, k)
    print('Calculating distances within buckets...')

    with Pool(psutil.cpu_count(logical=False)) as p:
        do = lambda x: sparse_dist(*x)
        work = [([x], k, None) for x in hash_buckets]
        D = {}
        with tqdm(total=len(hash_buckets)) as pbar:
            for d in p.istarmap(sparse_dist, work):
                pbar.update()
                D.update(d)
        p.close()

    cm = sparse_view(barcodes, D)

    print(f'Selecting barcodes with minimum edit distance {k}...')
    group_ids = [0] * len(barcodes)
    selected = maxy_clique_groups(cm, group_ids)

    selected_barcodes = [barcodes[x] for x in selected]
    print(f'Selected {len(selected_barcodes)} barcodes')
    
    return (df_bcs
            .query('barcode == @selected_barcodes')
            .assign(n=n, k=k, homopolymer=homopolymer, 
                gc_min=gc_min, gc_max=gc_max))

In [51]:
def DNA_Barcode_Generator_for_JSB(barcode_length, 
                          desired_Levenshtein_distance, 
                          sequence_to_exclude,
                          left_seq, 
                          right_seq, 
                          max_MM_to_exclude,
                          file_suffix,
                          barcode_length_cap = 11, 
                          min_GC_percent = 30, 
                          max_GC_percent = 70,
                          max_homopolymer_length = 4,
                          seq_input_number_limit = 0,
                          max_pairs_checked = int(1e6)):

    
    
    #Set variables
    limit = None if seq_input_number_limit == 0 else seq_input_number_limit
    exclude = None if sequence_to_exclude == '' else sequence_to_exclude
    
    #Run main function
    df_bcs = create_barcode_set_for_JSB(barcode_length, desired_Levenshtein_distance, max_homopolymer_length,
                                        min_GC_percent/100, max_GC_percent/100, left_seq, right_seq,
                                        max_MM_to_exclude, barcode_length_cap, exclude, limit)
    
    #Validate output
    failures = handle_failures(df_bcs['barcode'], desired_Levenshtein_distance, max_pairs_checked)

    #Export barcodes to .csv
    failure_tag = '.failure' if failures else ''
    filename = f'barcodes_n{barcode_length}_k{desired_Levenshtein_distance}{failure_tag}_{file_suffix}.csv'
    df_bcs.to_csv(filename, index=None)

## Current Use Case

In [59]:
DNA_Barcode_Generator(barcode_length, desired_Levenshtein_distance, regex_for_exclusion, file_suffix,
                      barcode_length_cap, min_GC_percent, max_GC_percent, max_homopolymer_length,
                      seq_input_number_limit, max_pairs_checked)

Generated 16384 barcodes of length 7
Retained 8832 barcodes after filtering for GC content and homopolymers
Retained 2341 barcodes after removing matches to "(?:(GTC$|GTCC|GT$|^TCC)){s<2}"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 98 barcodes
Validating barcodes...
All barcodes passed!


In [60]:
DNA_Barcode_Generator_for_JSB(barcode_length, desired_Levenshtein_distance, sequence_to_exclude, left_seq, 
                          right_seq, max_MM_to_exclude, file_suffix, barcode_length_cap, min_GC_percent, 
                          max_GC_percent, max_homopolymer_length, seq_input_number_limit, max_pairs_checked)

Generated 16384 barcodes of length 7
Retained 8832 barcodes after filtering for GC content and homopolymers
Retained 2341 barcodes after removing matches to "(GTCC){s<2}"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 98 barcodes
Validating barcodes...
All barcodes passed!


## LAMP-seq Use Case

Primer Sets: 
- Primer Set 1: TGCGGCCAATGTTTGTAATCAG NNNNNNNNNN CCAAGGAAATTTTGGGGAC, excluding GTCC with 1 mismatch as before
- Primer Set 2: ACCACGAAAGCAAGAAAAAGAAG NNNNNNNNNN TTCGTTTCGGAAGAGACAG, excluding CTGT with 1 mismatch as before
- Primer Set 3: GAGCCACACGCAGCTCATTGTA NNNNNNNNNN TCACCAACTGGGACGACA, excluding TGTC with 1 mismatch as before

In [10]:
DNA_Barcode_Generator(10, 3, "(?:(GTC$|GTCC|GT$|^TCC)){s<2}", "PrimerSet1")

Generated 1048576 barcodes of length 10
Retained 653896 barcodes after filtering for GC content and homopolymers
Retained 145221 barcodes after removing matches to "(?:(GTC$|GTCC|GT$|^TCC)){s<2}"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 3047 barcodes
Validating barcodes...
Checked 1,000,000 barcode pairs, all passed!


In [13]:
DNA_Barcode_Generator(10, 3, "(?:(CTGT|CTG$)){s<2}|(?:(^TGT|CT$))", "PrimerSet2")

Generated 1048576 barcodes of length 10
Retained 653896 barcodes after filtering for GC content and homopolymers
Retained 324674 barcodes after removing matches to "(?:(CTGT|CTG$)){s<2}|(?:(^TGT|CT$))"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 5726 barcodes
Validating barcodes...
Checked 1,000,000 barcode pairs, all passed!


In [14]:
DNA_Barcode_Generator(10, 3, "(?:(TGTC|TG$)){s<2}|(?:(^TC|TGT$))", "PrimerSet3")

Generated 1048576 barcodes of length 10
Retained 653896 barcodes after filtering for GC content and homopolymers
Retained 226533 barcodes after removing matches to "(?:(TGTC|TG$)){s<2}|(?:(^TC|TGT$))"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 4194 barcodes
Validating barcodes...
Checked 1,000,000 barcode pairs, all passed!


## LAMP-seq Use Case of JSB Variant 

In [53]:
DNA_Barcode_Generator_for_JSB(barcode_length = 7, #Smaller than 10 because faster to run
                          desired_Levenshtein_distance = 3, 
                          sequence_to_exclude = "GTCC",
                          left_seq = "TCAG", 
                          right_seq = "CCAA", 
                          max_MM_to_exclude = 1,
                          file_suffix = "PrimerSet1JSB")

Generated 16384 barcodes of length 7
Retained 8832 barcodes after filtering for GC content and homopolymers
Retained 2341 barcodes after removing matches to "(GTCC){s<2}"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 3...
Selected 98 barcodes
Validating barcodes...
All barcodes passed!


## Deprecated Test Cases

In [15]:
create_barcode_set(7, 2, 4, 0.3, 0.7, exclude=None,
    limit=None)

Generated 16384 barcodes of length 7
Retained 8832 barcodes after filtering for GC content and homopolymers
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 2...
Selected 1991 barcodes


,barcode,gc,n,k,homopolymer,gc_min,gc_max
71,AAACACG,0.428571,7,2,4,0.3,0.7
77,AAACAGC,0.428571,7,2,4,0.3,0.7
81,AAACCAC,0.428571,7,2,4,0.3,0.7
84,AAACCCA,0.428571,7,2,4,0.3,0.7
91,AAACCTG,0.428571,7,2,4,0.3,0.7
...,...,...,...,...,...,...,...
16162,GGGATAT,0.428571,7,2,4,0.3,0.7
16168,GGGATTA,0.428571,7,2,4,0.3,0.7
16258,GGGTAAT,0.428571,7,2,4,0.3,0.7
16264,GGGTATA,0.428571,7,2,4,0.3,0.7


In [16]:
create_barcode_set(7, 2, 4, 0.3, 0.7, exclude="AAAC",
    limit=None)

Generated 16384 barcodes of length 7
Retained 8832 barcodes after filtering for GC content and homopolymers
Retained 8717 barcodes after removing matches to "AAAC"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 2...
Selected 1979 barcodes


,barcode,gc,n,k,homopolymer,gc_min,gc_max
151,AAATCCG,0.428571,7,2,4,0.3,0.7
157,AAATCGC,0.428571,7,2,4,0.3,0.7
181,AAATGCC,0.428571,7,2,4,0.3,0.7
191,AAATGGG,0.428571,7,2,4,0.3,0.7
197,AAAGACC,0.428571,7,2,4,0.3,0.7
...,...,...,...,...,...,...,...
16162,GGGATAT,0.428571,7,2,4,0.3,0.7
16168,GGGATTA,0.428571,7,2,4,0.3,0.7
16258,GGGTAAT,0.428571,7,2,4,0.3,0.7
16264,GGGTATA,0.428571,7,2,4,0.3,0.7


In [17]:
create_barcode_set(7, 2, 4, 0.5, 0.7, exclude="AAAC",
    limit=None)

Generated 16384 barcodes of length 7
Retained 4416 barcodes after filtering for GC content and homopolymers
Retained 4385 barcodes after removing matches to "AAAC"
Assigning barcodes to hash buckets...
Calculating distances within buckets...



Selecting barcodes with minimum edit distance 2...
Selected 2193 barcodes


,barcode,gc,n,k,homopolymer,gc_min,gc_max
215,AAAGCCG,0.571429,7,2,4,0.5,0.7
221,AAAGCGC,0.571429,7,2,4,0.5,0.7
245,AAAGGCC,0.571429,7,2,4,0.5,0.7
277,AACACCC,0.571429,7,2,4,0.5,0.7
287,AACACGG,0.571429,7,2,4,0.5,0.7
...,...,...,...,...,...,...,...
16294,GGGTTCT,0.571429,7,2,4,0.5,0.7
16299,GGGTTTG,0.571429,7,2,4,0.5,0.7
16300,GGGTTGA,0.571429,7,2,4,0.5,0.7
16306,GGGTGAT,0.571429,7,2,4,0.5,0.7
